<a href="https://colab.research.google.com/github/smitithaparDartmouth/ENGS-108-Fall-23-assignment_2_Fall2023--Smiti-Thapar-/blob/main/Assignment_6_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENGS 108 Fall 2023 Assignment 6**

*Due November 14, 2023 at 11:59PM on Github*

**Instructors:** George Cybenko

**TAs:** Sunishka Jain, Diksha Bubna, Ganesh Rohit Nirogi , Kushagra Rawat


---

## **Rules and Requirements**


1.   You are only allowed to use Python packages that are explicity imported in
the assignment notebook or are standard (bultin) python libraries like random, os, sys, etc, (Standard Bultin Python libraries will have a Python.org documentation). For this assignment you may use:
  *   [numpy](https://numpy.org/doc/stable/)
  *   [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  *   [scikit-learn](https://scikit-learn.org/stable/)
  *   [matplotlib](https://matplotlib.org/)
  *   [tensorflow](https://www.tensorflow.org/)

2.   All code must be fit into the designated code or text blocks in the assignment notebook. They are indentified by a **TODO** qualifier.

3. For analytical questions that don't require code, type your answer cleanly in Markdown. For help, see the [Google Colab Markdown Guide](https://colab.research.google.com/notebooks/markdown_guide.ipynb).

---


In [1]:
''' Import Statements '''
import os
import random
import numpy as np
import pandas as pd
import csv
import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Creating a Trump Tweet Generator!!!
Finally it's happened. I am allowed to make my dream assignment. Trust me it's the best assignment. People tell me all the time how they wish they could have had an assignment as great as this. You will see why! But first some housekeeping...

# Data Loading
In this assignment we will be using a repository of Donald Trump tweets scrapped from Twitter through June 2020 from [Kaggle](https://www.kaggle.com/datasets/austinreese/trump-tweets) and will use the following code blocks to download the dataset directly to your Google Drive.

## Creating a Kaggle API Token
First we will need to download an API token from Kaggle in order to download the dataset, so our first step is to create a Kaggle account if you don't already have one. (You should have done this in Assignment 4 in case this looks familiar.
1. Create a Kaggle account by following the sign up instructions [here](https://www.kaggle.com/).
2. Log into your Kaggle account and click your account icon on the upper righthand side.
3. Then select **Account** from the dropdown/sidebar menu.
4. Scroll down to the **API** section and select **Create New API Token**.
5. This will download a JSON file called kaggle.json to your Downloads folder on your computer.
6. Now run the following code block and when the **Browse** button appears, click it and select that kaggle.json file.

In [2]:
# Run this code block after creating a Kaggle API token as instructed above.
! pip install -q kaggle
from google.colab import files

# Will ask you to upload kaggle.json file and remove any old ones.
if os.path.exists('kaggle.json'):
  os.remove('kaggle.json')
files.upload()

# Will create the appropriate directory structure
if not os.path.exists('/root/.kaggle'):
  ! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# Also we are going to make a directory called result
if not os.path.exists('/content/results'):
  ! mkdir /content/result

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/content/result’: File exists


## Downloading the Dataset

7. Now we have downloaded our Kaggle credentials we can now download the Trump Tweets Dataset (or any other Kaggle dataset for that matter) directly into our Google Drive.

In [3]:
! kaggle datasets download austinreese/trump-tweets
# Will check to see if the yoga postures zip file has been unzipped and will unzip the file if not.
! unzip trump-tweets.zip

trump-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  trump-tweets.zip
replace realdonaldtrump.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: realdonaldtrump.csv     
replace trumptweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: trumptweets.csv         


## Loading the Dataset using Pandas
Now let's inspect the trump tweets dataset and see what we have to work with... Brace yourselves.

In [4]:
# Let's load in the two files that we inflated from the Kaggle download. Both realdonaldtrump.csv and trumptweets.csv are the same.
real_donald_trump_df = pd.read_csv('realdonaldtrump.csv')

In [5]:
real_donald_trump_df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


## Problem 1: Pre-Processing the Tweets
As you may have noticed from the dataframe we just loaded, there are some special characters we need to handle. If we are making a tweet or sentence generator, we don't want to mess with special characters like commas or colons or really even captialization. So in the following section you are going to preprocess the data and strip these special characters out.

### Task 1: The Preprocess function
In the following code block complete the preprocess function that will strip or substitute out various special characters or sequences of characters that may not be ideal when training a sentence generator. We will use the built-in re python library to do a number of substitutions, and I have given you a skeleton below, see if you can complete it.

In [6]:
import re
REGEX_SUBS = {
  '\\\\': ' ',  # Use '\\\\' instead of '\\'
  '\n': ' ',
  '&': '',
  'RT ': '',
  '~': '',
  '#': '',
  '!+': '',
  r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)': 'link',  # Use raw string (prefix 'r') here
  r'\\\'': '',  # Corrected line
  r'[@]\w+': 'user',  # Use raw string (prefix 'r') here
  '[:|;]': '',
  r'[\\x]\W+|\d': '',  # Use raw string (prefix 'r') here
  r'[\\x]\w+': '',  # Use raw string (prefix 'r') here
  '    ': ' ',
  '   ': ' ',
  '  ': ' ',
  '@': ' '
}

def preprocess(text):
    for key, value in REGEX_SUBS.items():
        text = re.sub(key, value, text)
    text = text.lower()  # Make the text lowercase
    return text

### Task 2: Preprocess the dataset.
Now that we have our preprocess function, let's preprocess all the tweets.

In [7]:
#TODO: Run all the Dataframe content through your preprocess function.
real_donald_trump_df['content'] = real_donald_trump_df['content'].apply(preprocess)

In [8]:
real_donald_trump_df['content']

0        be sure to tune in and watch donald trump on l...
1        donald trump will be appearing on the view tom...
2        donald trump reads top ten financial tips on l...
3        new blog post celebrity apprentice finale and ...
4        "my persona will never be that of a wallflower...
                               ...                        
43347    joe biden was a total failure in government. h...
43348    will be interviewed on   seanhannity tonight a...
43349                                                 link
43350                                                 link
43351                                                 link
Name: content, Length: 43352, dtype: object

### Task 3: Tokenize the Data
The next step in every Natural Language Processing task is to tokenize the data, i.e., seperate our words, special characters, etc. into separate unique tokens. We will be using the [Natural Language Tool Kit](https://www.nltk.org/index.html) in python to accomplish this. Study the nltk API docs and see if you can tokenize our data.

In [9]:
from nltk.tokenize import word_tokenize
tokenized = [word_tokenize(tweet) for tweet in real_donald_trump_df['content']]


### Task 4: Build a Vocabulary
Now that we have our data tokenized, let's build a vocabulary including beginning and ending of sentence tokens, i.e., \<s>, \</s> for example. At this point let's also add in these beginning and end tokens into each of our data instances.

In [10]:
vocab_keys = {'<s>', '</s>'}
# Step 1: Extend vocab_keys with all unique tokens from tokenized data
for tweet_tokens in tokenized:
    vocab_keys.update(tweet_tokens)

# Step 2: Create vocabulary and inverse vocabulary dictionaries
# Rebuild the vocabulary to include all tokens
vocabulary = {token: idx for idx, token in enumerate(vocab_keys)}
vocabulary_inverse = {idx: token for token, idx in vocabulary.items()}

# Step 3: Encode training data
train_data = []
for tweet_tokens in tokenized:
    # Adding beginning and end tokens to each tokenized tweet
    tweet_tokens = ['<s>'] + tweet_tokens + ['</s>']
    # Converting tokens to their corresponding indices
    encoded_tweet = [vocabulary[token] for token in tweet_tokens]
    train_data.append(encoded_tweet)


In [11]:
vocabulary

{'damnit': 0,
 'cjtf–oir': 1,
 'commit': 2,
 'amalfas': 3,
 'amilcaralvarez': 4,
 'scorned': 5,
 'chernuna': 6,
 'khan': 7,
 'docs': 8,
 'lainey': 9,
 'abdeslam': 10,
 'alfonse': 11,
 'also': 12,
 'pbrunnerrr': 13,
 "'trumping": 14,
 'assemble': 15,
 'nydailynews': 16,
 'lanas': 17,
 'hindenburg': 18,
 'अधिक': 19,
 'bigpeter': 20,
 'best_is_me': 21,
 'mavelous': 22,
 'demonstrated': 23,
 'sirenasola': 24,
 'tijfbutler': 25,
 'bleed': 26,
 'kiernanproud': 27,
 'thai': 28,
 'helsinki': 29,
 'watered': 30,
 'bizpacreview': 31,
 'colle…': 32,
 'world-famous': 33,
 'judgement': 34,
 'daily_record': 35,
 'markwcarlson': 36,
 'tcloer': 37,
 'planes': 38,
 'swampman': 39,
 'erictrump-': 40,
 'celebrityapprentice': 41,
 'fpatterson': 42,
 'rohrabacher': 43,
 'spazosaurus': 44,
 'aleady': 45,
 'brain': 46,
 'd-calif': 47,
 'bike': 48,
 'wikileaks': 49,
 'floyd': 50,
 'oversplayer': 51,
 'minute': 52,
 'folk': 53,
 'c_archaeology': 54,
 'touch-': 55,
 'michaelpleahy': 56,
 'courageous': 57,
 'vid

# Problem 1: N-gram Language Model
In this problem, you will be building a couple n-gram language modeling and see how well just taking pure frequency counts and building a conditional probability distrbution will work.

## Task 1: A 2-gram (Bigram) Model
Recall that our goal in building a language model is to represent the conditional probability $P(w_i | w_{i-1})$ for pairs of words $w_i$ and $w_j$.

### Part A: Frequencies
Go through the encoded Trump tweets and calculate the frequencies of all words as well as all pair of words that appear next to each other in the corpus.

In [12]:
from collections import defaultdict

def make_grams(tweets, n=2):
    grams = []
    for tweet in tweets:
        # Add bigrams to the list
        for i in range(len(tweet) - n + 1):
            grams.append(tuple(tweet[i:i + n]))
    return grams
def calculate_counts(grams):
    # Counting frequencies of each gram
    counts = defaultdict(int)
    for gram in grams:
        counts[gram] += 1
    return counts


In [13]:
bigrams = make_grams(train_data, n=2)

# Calculate bigram frequencies
bigram_counts = calculate_counts(bigrams)

# Display the calculated counts
print(bigram_counts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
grams = make_grams(train_data)
counts = calculate_counts(grams)

### Part B: Probabilities
Now from the counts above we will calculate an associated conditional probabilities.

In [15]:
# probs = defaultdict(dict)
# #for gram, count in counts.items():
#   #TODO: Do something...
# x = 0

#   # Calculate probabilities for each bigram
# for bigram, count in bigram_counts.items():
#     # Extract the first and second word from the bigram
#     word1, word2 = bigram
#     x += 1
#     # Calculate the conditional probability P(word2 | word1)
#     probability = count / sum(bigram_counts[prev_word, current_word] for prev_word, current_word in bigram_counts if prev_word == word1)

#     # Store the probability in the dictionary
#     probs[word1][word2] = probability

# # Display the calculated probabilities
# print(probs)

unigram_counts = defaultdict(int)
for (word1, word2), count in bigram_counts.items():
    unigram_counts[word1] += count

# Now, calculate the conditional probabilities
probs = defaultdict(dict)
for (word1, word2), count in bigram_counts.items():
    probability = count / unigram_counts[word1]
    probs[word1][word2] = probability

### Part C: Make an Bigram Generator Function
Now that we have our probability distrbution, let's make a generator function so that we can generate random Trump tweets using our bigram language model.

In [16]:
# def get_next_word(next_word_probs):
#   #TODO: Write a function to get the next word based on the previous words conditional probabilities.
#   return something

# def generate(start_text='', n=10, probs=probs, vocabulary=vocabulary, vocabulary_inverse=vocabulary_inverse):
#   # Helper code to create the start text.
#   start_text = ['<s>'] + nltk.word_tokenize(preprocess(start_text))

#   #TODO: Encode and generate a trump tweet.
#   return something

import random

def get_next_word(next_word_probs):
    # Convert the probability distribution into a list of (word, prob) tuples
    words, probs = zip(*next_word_probs.items())
    # Randomly choose a word based on the provided probabilities
    next_word = random.choices(words, weights=probs, k=1)[0]
    return next_word

def generate(start_text='', n=10, probs=probs, vocabulary=vocabulary, vocabulary_inverse=vocabulary_inverse):
    # Tokenize and preprocess the start text
    start_text_tokens = ['<s>'] + nltk.word_tokenize(preprocess(start_text))

    # Initialize the generated text with the start text
    generated_text = start_text_tokens

    # Generate n words
    for _ in range(n):
        # Get the last word in the current text
        last_word = generated_text[-1]
        # Get the encoded version of the last word
        last_word_encoded = vocabulary.get(last_word, None)

        # If the last word is not in the vocabulary or there's no next word, break
        if last_word_encoded is None or last_word_encoded not in probs:
            break

        # Get the next word probabilities
        next_word_probs = probs[last_word_encoded]

        # Get the next word and decode it
        next_word_encoded = get_next_word(next_word_probs)
        next_word = vocabulary_inverse[next_word_encoded]

        # Append the next word to the generated text
        generated_text.append(next_word)

        # If the next word is the end token, break
        if next_word == '</s>':
            break

    # Join the words to form the final text
    return ' '.join(generated_text[1:])  # Exclude the start token from the final text


In [17]:
generate('I can make')

'i can make the same with wolfblitzer from highways to be really looking'

## **(Bonus)** Task 2: Make a Trigram Model
Using your code from Parts A-C, see if you can build a trigram (3-gram) model that might make tweets a little more logical.

In [18]:
#TODO: Your code goes here.

# Problem 2: Using a Transformer
In this section we are going to leverage a pretrained transformer, i.e., [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), built by the [Hugging Faces Team](https://huggingface.co/gpt2?text=A+long+time+ago%2C). We will also be using their tokenizers because they have been optimized for the language generation task. You should be aware that behind the scenes, their model is using [PyTorch](https://pytorch.org/), the deep learning library built by Facebook, and is quickly becoming more popular than our beloved Tensorflow.

## Task 1: Install and load the GPT2 Model

In [19]:
# Run the following code to install the HuggingFace's transformer python package.
! pip install transformers

In [20]:
# TODO: Follow the link above and load the GPT2 model as well as the tokenizer.
# NOTE: Replace GPT2Model with GPT2LMHeadModel (this is the model for finetuning use cases.)
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


## Task 2: Formatting our Training Data
Upcoming we will be fine-tunning this GPT-2 model on our Trump Tweets, but in order to leverage some of the utility classes built by HuggingFaces, we want to take our preprocessed Trump tweets and place them in a flat text file.

In [21]:
#TODO: Take our preprocessed trump tweets and write them to a text file
with open('trumpdata.txt', 'w', encoding='utf-8') as text_file:
    for tweet in real_donald_trump_df['content']:
        # Write each tweet followed by a newline character
        text_file.write(tweet + '\n')

## Task 4: Fine-tuning the Model
In the following sections, we will complete a couple functions that will allow us to fine-tune the GPT-2 model to our Trump Tweets. I am going to give you a couple of these helper functions, but leave you to write parts of the training function. See the following documentation from Hugging Faces to see the attributes for the [Trainer Class](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer).

In [22]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size = 128):
    # Will load and tokenize the data
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    # Helper Function
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

def train(
    train_file_path,
    model,
    tokenizer,
    output_dir,
    overwrite_output_dir,
    num_train_epochs,
    save_steps
    ):
    # Load the dataset
    dataset = load_dataset(train_file_path, tokenizer)

    # Load the data collator
    data_collator = load_data_collator(tokenizer, mlm=False)

    # Initialize TrainingArguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=8,
        num_train_epochs=num_train_epochs,
        save_steps=save_steps,
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    # Train the model
    trainer.train()

    # Save the model and tokenizer
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

In [23]:
#TODO: Set necessary parameters, here are some defaults.
train_file_path = "/content/trumpdata.txt"
output_dir = '/content/result'
overwrite_output_dir = False
num_train_epochs = 1
save_steps = 500

In [24]:
!pip install accelerate -U


In [25]:
#TODO: Use your train function to train the model. It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model=model,
    tokenizer=tokenizer,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,3.852200
1000,3.577700


## Task 5: Creating a Tweet Generator
Now that we have our trained model, it's time to generate some Tweets. Since we should have saved our model and tokenizer to an output directory, I've already made some helper functions to load those in. We will focus on our *generate_text* function. The function will take as input some start text like "I am" or "My country is", etc., as well as a max_length parameter which tells the model how much text to generate. Let's Go!

In [30]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(sequence, max_length):
    # Load in the finetuned model and tokenizer
    model = load_model('/content/result')  # Assuming this is the model path
    tokenizer = load_tokenizer('/content/result')  # Assuming this is the tokenizer path

    # Encode our passed sequence
    ids = tokenizer.encode(sequence, return_tensors='pt')

    # Use the generate method in our model class to generate output tokens
    final_outputs = model.generate(
        input_ids=ids,
        max_length=max_length,
        do_sample=True,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )

    # Function to print and decode output
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [31]:
#TODO: Now generate some text!
generate_text('I can make', 30)

I can make my own mistakes, and sometimes these are mistakes, but my life is much better with a smile than the u.s.a.
